In [1]:
import pandas as pd
import os

# Read all CSV files in the 'csv_results' folder and concatenate them into a single DataFrame
folder = 'ablation_study'
df = pd.concat([pd.read_csv(os.path.join(folder, f)).assign(run=f.split('.')[0][17:]) for f in os.listdir(folder) if f.endswith('.csv')], ignore_index=True)


# Extract best result/run for each task
func = {
    'anomaly_detection': max,
    'imputation': min,
    'long_term_forecast': min,
    'short_term_forecast': min,
    'classification': max,
}

# Construct best dataframe
best_df = pd.DataFrame(columns=df.columns)
for task in df['task_name'].unique():
    df_task = df[df['task_name'] == task]
    fun = func[df_task['task_type'].values[0]]
    best_val, best_run = fun(zip(df_task['main_metric'], df_task['run']))
    best_df = pd.concat([best_df, df_task[df_task['run'] == best_run]], ignore_index=True)

best_df

,task_type,task_name,dataset,main_metric,accuracy,f_score,full_task_name,horizon,mae,mape_average,...,owa_quarterly,owa_yearly,precision,recall,smape_average,smape_monthly,smape_others,smape_quarterly,smape_yearly,run
0,anomaly_detection,anomaly_detection_SMD,SMD,0.789600,0.981100,0.7896,NaN,NaN,NaN,NaN,...,NaN,NaN,0.7349,0.8531,NaN,NaN,NaN,NaN,NaN,run_est_fixed_lr
1,anomaly_detection,anomaly_detection_PSM,PSM,0.925500,0.961200,0.9255,NaN,NaN,NaN,NaN,...,NaN,NaN,0.9907,0.8683,NaN,NaN,NaN,NaN,NaN,run_est_fixed_lr
2,anomaly_detection,anomaly_detection_MSL,MSL,0.819900,0.965300,0.8199,NaN,NaN,NaN,NaN,...,NaN,NaN,0.9065,0.7484,NaN,NaN,NaN,NaN,NaN,run_est_fixed_lr
3,anomaly_detection,anomaly_detection_SMAP,SMAP,0.732200,0.942300,0.7322,NaN,NaN,NaN,NaN,...,NaN,NaN,0.9023,0.6161,NaN,NaN,NaN,NaN,NaN,run_est_fixed_lr
4,anomaly_detection,anomaly_detection_SWAT,SWAT,0.835200,0.965200,0.8352,NaN,NaN,NaN,NaN,...,NaN,NaN,0.9838,0.7256,NaN,NaN,NaN,NaN,NaN,run_est_fixed_lr
5,imputation,imputation_ETTh1_mask_0.125,ETTh1,0.074751,NaN,NaN,NaN,NaN,0.194912,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,run_est_fixed_lr
6,imputation,imputation_ETTh1_mask_0.25,ETTh1,0.132026,NaN,NaN,NaN,NaN,0.265849,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,run_est_fixed_lr
7,imputation,imputation_weather_mask_0.375,weather,0.057910,NaN,NaN,NaN,NaN,0.130782,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,run_est_fixed_lr
8,imputation,imputation_ECL_mask_0.125,ECL,0.186551,NaN,NaN,NaN,NaN,0.303792,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,run_est_fixed_lr
9,imputation,imputation_ETTh1_mask_0.375,ETTh1,0.161126,NaN,NaN,NaN,NaN,0.289891,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,run_est_fixed_lr


In [2]:
best_df.run.value_counts()

run_est_fixed_lr    59
Name: run, dtype: int64

In [3]:
best_df.groupby('task_type')['main_metric'].mean()

task_type
anomaly_detection      0.820480
classification         0.715748
imputation             0.135041
long_term_forecast     1.112920
short_term_forecast    1.079000
Name: main_metric, dtype: float64

In [6]:
best_df.groupby('task_type')['run'].value_counts()

task_type            run   
anomaly_detection    run_8      2
                     run_10     1
                     run_2      1
                     run_5      1
classification       run_10     3
                     run_5      2
                     run_3      1
                     run_4      1
                     run_6      1
                     run_7      1
                     run_9      1
imputation           run_1      6
                     run_5      3
                     run_10     1
                     run_4      1
                     run_9      1
long_term_forecast   run_6     13
                     run_7      6
                     run_9      6
                     run_2      4
                     run_4      3
                     run_1      1
                     run_5      1
short_term_forecast  run_10     4
                     run_2      1
                     run_9      1
Name: run, dtype: int64

In [21]:
runs = {
    'run_1': {
        'layer': 1,
        'memory_units': 4,
        'memory_dim': 100,
        'model_dim': 64,
        'memory_connectivity': 0.05
    },
    'run_2': {
        'layer': 2,
        'memory_units': 4,
        'memory_dim': 128,
        'model_dim': 128,
        'memory_connectivity': 0.05
    },
    'run_3': {
        'layer': 4,
        'memory_units': 8,
        'memory_dim': 64,
        'model_dim': 32,
        'memory_connectivity': 0.2
    },
    'run_4': {
        'layer': 1,
        'memory_units': 2,
        'memory_dim': 512,
        'model_dim': 128,
        'memory_connectivity': 0.025
    },
    'run_5': {
        'layer': 1,
        'memory_units': 4,
        'memory_dim': 128,
        'model_dim': 64,
        'memory_connectivity': 0.05,
        # 'normalisation_forecast': False
    },
    'run_6': {
        'layer': 2,
        'memory_units': 8,
        'memory_dim': 64,
        'model_dim': 64,
        'memory_connectivity': 0.1,
        # 'bug_normalisation_short_term_forecast': True
    },
    'run_7': {
        'layer': 3,
        'memory_units': 4,
        'memory_dim': 32,
        'model_dim': 64,
        'memory_connectivity': 0.25,
        # 'bug_normalisation_short_term_forecast': True
    },
    'run_8': {
        'layer': 4,
        'memory_units': 2,
        'memory_dim': 64,
        'model_dim': 128,
        'memory_connectivity': 0.125
    },
    'run_9': {
        'layer': 2,
        'memory_units': 16,
        'memory_dim': 32,
        'model_dim': 32,
        'memory_connectivity': 0.25,
        # 'no_long_term_forecast': True
    },
    'run_10': {
        'layer': 1,
        'memory_units': 16,
        'memory_dim': 64,
        'model_dim': 64,
        'memory_connectivity': 0.125,
        # 'no_long_term_forecast': True
    },
}

rdf = best_df[['task_type', 'task_name', 'run', 'main_metric']]
rdf['parameters'] = rdf['run'].map(runs)
rdf

/var/folders/_8/z8rcf_ns4k9f2rw5zchj85mw0000gn/T/ipykernel_18138/1390223157.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdf['parameters'] = rdf['run'].map(runs)


,task_type,task_name,run,main_metric,parameters
0,anomaly_detection,anomaly_detection_SMD,run_2,0.791500,"{'layer': 2, 'memory_units': 4, 'memory_dim': ..."
1,anomaly_detection,anomaly_detection_PSM,run_10,0.930500,"{'layer': 1, 'memory_units': 16, 'memory_dim':..."
2,anomaly_detection,anomaly_detection_MSL,run_5,0.822400,"{'layer': 1, 'memory_units': 4, 'memory_dim': ..."
3,anomaly_detection,anomaly_detection_SMAP,run_8,0.829400,"{'layer': 4, 'memory_units': 2, 'memory_dim': ..."
4,anomaly_detection,anomaly_detection_SWAT,run_8,0.888900,"{'layer': 4, 'memory_units': 2, 'memory_dim': ..."
...,...,...,...,...,...
62,long_term_forecast,long_term_forecast_weather_336,run_6,0.284781,"{'layer': 2, 'memory_units': 8, 'memory_dim': ..."
63,long_term_forecast,long_term_forecast_weather_720,run_5,0.410082,"{'layer': 1, 'memory_units': 4, 'memory_dim': ..."
64,long_term_forecast,long_term_forecast_ETTm2_720,run_9,0.438327,"{'layer': 2, 'memory_units': 16, 'memory_dim':..."
65,long_term_forecast,long_term_forecast_Exchange_720,run_6,0.904805,"{'layer': 2, 'memory_units': 8, 'memory_dim': ..."


In [20]:
for task_type in rdf['task_type'].unique():
    print(f"\n{task_type.upper()}")
    for task in rdf[rdf['task_type'] == task_type]['task_name'].unique():
        print(f"- {task} : {rdf[rdf['task_name'] == task]['parameters'].values[0]}")


ANOMALY_DETECTION
- anomaly_detection_SMD : {'layer': 2, 'memory_units': 4, 'memory_dim': 128, 'model_dim': 128, 'memory_connectivity': 0.05}
- anomaly_detection_PSM : {'layer': 1, 'memory_units': 16, 'memory_dim': 64, 'model_dim': 64, 'memory_connectivity': 0.125}
- anomaly_detection_MSL : {'layer': 1, 'memory_units': 4, 'memory_dim': 128, 'model_dim': 64, 'memory_connectivity': 0.05}
- anomaly_detection_SMAP : {'layer': 4, 'memory_units': 2, 'memory_dim': 64, 'model_dim': 128, 'memory_connectivity': 0.125}
- anomaly_detection_SWAT : {'layer': 4, 'memory_units': 2, 'memory_dim': 64, 'model_dim': 128, 'memory_connectivity': 0.125}

IMPUTATION
- imputation_ETTh1_mask_0.125 : {'layer': 1, 'memory_units': 4, 'memory_dim': 128, 'model_dim': 64, 'memory_connectivity': 0.05}
- imputation_ETTh1_mask_0.25 : {'layer': 1, 'memory_units': 4, 'memory_dim': 100, 'model_dim': 64, 'memory_connectivity': 0.05}
- imputation_ECL_mask_0.125 : {'layer': 1, 'memory_units': 4, 'memory_dim': 128, 'model_dim

In [ ]:
for task_type in rdf['task_type'].unique():
    print(f"\n{task_type.upper()}")
    for task in rdf[rdf['task_type'] == task_type]['task_name'].unique():
        print(f"- {task} : {rdf[rdf['task_name'] == task]['main_metric'].values[0]}")


ANOMALY_DETECTION
- anomaly_detection_SMD : 0.7915
- anomaly_detection_PSM : 0.9305
- anomaly_detection_MSL : 0.8224
- anomaly_detection_SMAP : 0.8294
- anomaly_detection_SWAT : 0.8889

IMPUTATION
- imputation_ETTh1_mask_0.125 : 0.0820519626140594
- imputation_ETTh1_mask_0.25 : 0.1327738761901855
- imputation_ECL_mask_0.125 : 0.188336431980133
- imputation_ETTh1_mask_0.375 : 0.1676205694675445
- imputation_ETTh1_mask_0.5 : 0.2119920253753662
- imputation_weather_mask_0.125 : 0.069200150668621
- imputation_ECL_mask_0.25 : 0.1939822435379028
- imputation_ECL_mask_0.375 : 0.2001051902770996
- imputation_ECL_mask_0.5 : 0.2036992907524109
- imputation_weather_mask_0.25 : 0.0403170250356197
- imputation_weather_mask_0.375 : 0.0530886836349964
- imputation_weather_mask_0.5 : 0.0643154606223106

LONG_TERM_FORECAST
- long_term_forecast_ili_24 : 2.6678621768951416
- long_term_forecast_ili_36 : 2.48522686958313
- long_term_forecast_Exchange_96 : 0.1174870282411575
- long_term_forecast_traffic_96